In [2]:
#Import necessary modules inlcuded Tensorflow
import pandas as pd
import tensorflow as tf
#import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
#Read in File
data = pd.read_csv("balanced_pop_unpop_features.csv")
data.head()


,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label
0,#Beautiful,0.21400,0.666,178242,0.677,0.000000,2,0.0979,-5.743,0.0326,100.014,4,0.178,1
1,#SELFIE,0.01340,0.807,183750,0.916,0.000012,0,0.0787,-3.282,0.2260,127.973,4,0.651,1
2,#thatPOWER,0.00162,0.791,279507,0.615,0.000065,6,0.0812,-6.149,0.0667,128.017,4,0.393,1
3,0 To 100 / The Catch Up,0.76300,0.707,275227,0.709,0.000000,11,0.2740,-3.979,0.3400,89.094,4,0.501,1
4,1-800-273-8255,0.57000,0.629,250173,0.572,0.000000,5,0.1920,-7.733,0.0387,100.015,4,0.386,1


In [27]:
#Remove track name column
data = data[["danceability","label"]]

In [28]:
#Shuffle dataset
for i in range(10): data = data.sample(frac=1)

In [29]:
#About a 70-30 split between training and testing 
train = data.iloc[:750]
test = data.iloc[750:]

In [30]:
train.head()

,danceability,label
25,0.579,1
1062,0.599,0
229,0.665,1
68,0.611,1
82,0.478,1


In [31]:
print(train.shape,test.shape)

(750, 2) (322, 2)


In [33]:
#Split entries in train_x: features and train_y: labels
train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1:].astype(int)

test_x = test.iloc[:,:-1]
test_y = test.iloc[:,-1:].astype(int)

#Take Z-score
train_x = (train_x-train_x.mean())/(train_x.std())
test_x = (test_x-test_x.mean())/(test_x.std())

In [34]:
train_x.head()

,danceability
25,-0.480682
1062,-0.338720
229,0.129753
68,-0.253543
82,-1.197587


In [35]:
#Create feature column for net 
feature_columns = []
for i in train_x.columns:
    feature_columns.append(tf.feature_column.numeric_column(key=i))

In [36]:
#Lil preview 
for i in feature_columns:
    print(i)
    print("")

_NumericColumn(key='danceability', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)



In [37]:
#Make estimator/ Model
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns
                                        , hidden_units=[5],
                                        n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2lt9yem4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11637ed30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [39]:
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y,300),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2lt9yem4/model.ckpt.
INFO:tensorflow:loss = 207.75041, step = 1
INFO:tensorflow:global_step/sec: 520.532
INFO:tensorflow:loss = 205.9241, step = 101 (0.193 sec)
INFO:tensorflow:global_step/sec: 807.689
INFO:tensorflow:loss = 207.8351, step = 201 (0.124 sec)
INFO:tensorflow:global_step/sec: 845.074
INFO:tensorflow:loss = 208.07153, step = 301 (0.118 sec)
INFO:tensorflow:global_step/sec: 852.913
INFO:tensorflow:loss = 207.4996, step = 401 (0.117 sec)
INFO:tensorflow:global_step/sec: 817.603
INFO:tensorflow:loss = 206.47284, step = 501 (0.123 sec)
INFO:tensorflow:global_step/sec: 797.868
INFO:tensorflow:loss = 207.75072, step = 601 (0.125 sec)
INFO:tensorflow:

In [40]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [41]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 3000))



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-15-04:46:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2lt9yem4/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-15-04:46:03
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.54037267, accuracy_baseline = 0.5124223, auc = 0.5671685, auc_precision_recall = 0.4243789, average_loss = 0.6885094, global_step = 1000, label/mean = 0.48757765, loss = 221.70003, precision = 0.5235602, prediction/mean = 0.5045379, recall = 0.6369427


In [42]:
eval_result

{'accuracy': 0.54037267,
 'accuracy_baseline': 0.5124223,
 'auc': 0.5671685,
 'auc_precision_recall': 0.4243789,
 'average_loss': 0.6885094,
 'global_step': 1000,
 'label/mean': 0.48757765,
 'loss': 221.70003,
 'precision': 0.5235602,
 'prediction/mean': 0.5045379,
 'recall': 0.6369427}

In [43]:
test_x.head()

,danceability
165,-1.445986
282,0.927423
272,1.024756
680,0.957372
660,1.369162


In [44]:
#Look at results of net preidctions compared to actual label
test_x_np = test_x.values.transpose()


predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=dict(test_x),
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

for i,j in zip(predicted_classes,test_y.label):
    print(i,j)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/81/_01kvw1s7xj6hjfw2y34w9_40000gn/T/tmp2lt9yem4/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[b'1'] 1
[b'1'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'1'] 0
[b'0'] 0
[b'0'] 0
[b'1'] 0
[b'1'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'1'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'1'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'1'] 1
[b'0'] 0
[b'1'] 1
[b'0'] 1
[b'1'] 1
[b'1'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'1'] 1
[b'1'] 0
[b'0'] 0
[b'1'] 1
[b'1'] 0
[b'1'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'0'] 0
[b'0'] 0
[b'0'] 1
[b'1'] 0
[b'0

/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [45]:
#A few more nets